In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

tf.random.set_seed(42)

class MyLSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
        super(MyLSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units,
                                       return_sequences=True,
                                       return_state=True,
                                       stateful=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, training=None, initial_state=None):
        x = self.embedding(inputs)

        x, h, c = self.lstm(x, initial_state=initial_state)

        output = self.dense(x)

        if training:
            return output
        return output, [h, c]

    def reset_states(self):
        self.lstm.reset_states()

class TextProcessor:
    def __init__(self, filepath, encoding='utf-8', seq_length=100):
        self.text = self._load_data(filepath, encoding)
        self.vocab = sorted(list(set(self.text)))
        self.vocab_size = len(self.vocab)
        self.seq_length = seq_length

        self.char2idx = {u: i for i, u in enumerate(self.vocab)}
        self.idx2char = np.array(self.vocab)

        print(f'Текст загружен. Длина: {len(self.text)} симв.')
        print(f'Уникальных символов: {self.vocab_size}')

    def _load_data(self, path, encoding):
        with open(path, 'r', encoding=encoding) as f:
            return f.read()

    def create_dataset(self, batch_size=64, buffer_size=10000):
        text_as_int = np.array([self.char2idx[c] for c in self.text])
        char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
        sequences = char_dataset.batch(self.seq_length + 1, drop_remainder=True)

        def split_input_target(chunk):
            return chunk[:-1], chunk[1:]

        dataset = sequences.map(split_input_target)
        dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

        dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return dataset

class Agent:
    def __init__(self, processor):
        self.processor = processor
        self.model = None
        self.embedding_dim = 256
        self.rnn_units = 1024
        self.checkpoint_dir = './training_checkpoints'

        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)

    def compile_model(self, batch_size):
        model = MyLSTMModel(
            vocab_size=self.processor.vocab_size,
            embedding_dim=self.embedding_dim,
            rnn_units=self.rnn_units,
            batch_size=batch_size
        )

        dummy_input = tf.zeros((batch_size, 1), dtype=tf.int32)
        model(dummy_input, training=False)

        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(optimizer='adam', loss=loss)
        return model

    def train(self, dataset, epochs=10):
        self.model = self.compile_model(batch_size=64)
        print("Начинаю обучение...")

        checkpoint_path = os.path.join(self.checkpoint_dir, "ckpt_{epoch}.weights.h5")
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=True,
            verbose=1
        )

        history = self.model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])
        return history

    def generate_text(self, start_string, num_generate=1000, temperature=1.0):
        gen_model = self.compile_model(batch_size=1)

        gen_model.set_weights(self.model.get_weights())

        gen_model.reset_states()

        input_eval = [self.processor.char2idx[s] for s in start_string]
        input_eval = tf.expand_dims(input_eval, 0)

        text_generated = []
        states = None

        print(f"Генерация {num_generate} символов...")
        for i in range(num_generate):
            predictions, states = gen_model(input_eval, training=False, initial_state=states)

            predictions = tf.squeeze(predictions, 0)

            predictions = predictions / temperature

            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(self.processor.idx2char[predicted_id])

        return start_string + ''.join(text_generated)

if __name__ == '__main__':
    try:
        proc = TextProcessor('input.txt', encoding='utf-8')
        train_data = proc.create_dataset(batch_size=64)

        agent = Agent(proc)

        agent.train(train_data, epochs=20)

        result = agent.generate_text(start_string=u"Читатели ", num_generate=1000, temperature=0.6)

        print("\n" + "="*50)
        print("РЕЗУЛЬТАТ ГЕНЕРАЦИИ:")
        print("="*50)
        print(result)

        # Сохранение
        with open('generated_result.txt', 'w', encoding='utf-8') as f:
            f.write(result)

    except FileNotFoundError:
        print("error")

Текст загружен. Длина: 692221 симв.
Уникальных символов: 151
Начинаю обучение...
Epoch 1/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 3.5539
Epoch 1: saving model to ./training_checkpoints/ckpt_1.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - loss: 3.5495
Epoch 2/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 2.5899
Epoch 2: saving model to ./training_checkpoints/ckpt_2.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - loss: 2.5895
Epoch 3/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 2.4179
Epoch 3: saving model to ./training_checkpoints/ckpt_3.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - loss: 2.4176
Epoch 4/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 2.2827
Epoch 4: saving model to ./training_checkpoints/ckpt_4.weights.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - loss: 2.2824
Epoch 5/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 2.1614
Epoch 5: saving model to ./training_checkpoints/ckpt_5.weights.h5
107/107 ━━━━━━━━━

In [1]:
import tensorflow as tf
print("Доступные GPU:", tf.config.list_physical_devices('GPU'))

Доступные GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
